# Introduction 

This is code for the plots in the report

# Imports 

In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [ ]:
activations = {}
def hook_fn(module, input, output):
    activations[module] = output

# Load data

In [ ]:
baseline_train_loss = torch.load("results/baseline/train_loss.pt")
baseline_test_loss = torch.load("results/baseline/test_loss.pt")
baseline_predictions = torch.load("results/baseline/predictions.pt")

baseline_activations = pickle.load(open("results/baseline_with_hooks/activations.pkl", "rb"))

final_predictions = torch.load("results/final_model/predictions.pt")
final_activations = pickle.load(open("results/final_model/activations.pkl", "rb"))

baseline_true_edges = torch.load("plot_helpers/train_test_val_edge_weights.pt")
final_true_edges = torch.load("plot_helpers/train_test_edge_weights.pt")

In [ ]:
def plot_train_val_loss(axes, train_loss, validation_loss, grid = True):
    axes.plot(range(len(train_loss)), train_loss, label='Training Loss')
    axes.plot(range(len(validation_loss)), validation_loss, label='Validation Loss')
    axes.set_title("Training and Validation Losses")
    axes.set_xlabel("Epoch")
    axes.set_ylabel("Loss")
    axes.legend()
    axes.set_ylim(0, 40)
    axes.grid(grid)

def ratings_distribution(axes, predicted, actual):
    sns.kdeplot(predicted, color='blue', fill=True, label="Predicted", ax = axes)
    sns.kdeplot(actual, color='orange', fill=True, label="Actual", ax = axes)
    axes.set_title("Density of Predicted and\n Actual Ratings")
    axes.set_xlabel("Rating")
    axes.set_ylabel("Density")
    axes.set_xlim(0, 6.5)
    axes.legend()

def plot_activation_heatmap(axes, activation, title, cbar=False, ytick = False):
    sns.heatmap(activation, vmin=-100, vmax=100, annot=False, cmap="viridis", ax=axes, cbar=cbar)
    axes.set_title(title)
    if not ytick:
        axes.set_yticks(ticks=[], labels=[])

# Baseline plots

In [ ]:
# Train / validation loss
# density plot

fig, axes = plt.subplots(1, 2, figsize=(6.4, 3.5))
plot_train_val_loss(axes[0], baseline_train_loss[5:], baseline_test_loss[5:], False)
ratings_distribution(axes[1], baseline_predictions, baseline_true_edges)

plt.tight_layout()
plt.savefig(f"plots/baseline_plots_1.png", dpi = 300, bbox_inches='tight')

In [ ]:
baseline_activations_keys = list(baseline_activations.keys())
fig, axes = plt.subplots(1, 2, figsize=(6.4, 3.5))
plot_activation_heatmap(axes[0], baseline_activations[baseline_activations_keys[2]].detach().cpu().numpy(), "GNN Layer 1", False, True)
plot_activation_heatmap(axes[1], baseline_activations[baseline_activations_keys[3]].detach().cpu().numpy(), "GNN Layer 2", True, False)
fig.supxlabel("Features")
fig.supylabel("Users")
plt.tight_layout()

plt.tight_layout()
plt.savefig(f"plots/baseline_plots_2.png", dpi = 300, bbox_inches='tight')

# Final plots

In [ ]:
# density plot
final_activations_keys = list(final_activations.keys())
fig, axes = plt.subplots(1, 2, figsize=(6.4, 3.5))
ratings_distribution(axes[0], final_predictions, final_true_edges)
plot_activation_heatmap(axes[1], final_activations[final_activations_keys[3]].detach().cpu().numpy(), "GNN Layer 2", True, True)

plt.tight_layout()
plt.savefig(f"plots/final_plots_1.png", dpi = 300, bbox_inches='tight')